In [44]:
import psycopg2

In [52]:
db_name = 'clients'
user_name = 'postgres'
passw = 'wasser'
conn = psycopg2.connect(database = db_name, user = user_name, password = passw) 
cur = conn.cursor()

In [53]:
def creation():
        cur.execute("""
        CREATE TABLE IF NOT EXISTS clients(
            id SERIAL PRIMARY KEY,
            name VARCHAR (20) NOT NULL,
            surname VARCHAR (30),
            email VARCHAR(20) UNIQUE
        );
        CREATE TABLE IF NOT EXISTS phone_numbers(
            ph_id SERIAL PRIMARY KEY,
            client_id INTEGER REFERENCES clients(id),
            number INTEGER UNIQUE
        );
        """)
        conn.commit()

In [54]:
def new_client(cl_name, cl_surname, cl_email):
        cur.execute("""
        INSERT INTO clients(name,surname,email) VALUES (%s, %s, %s);
        """, (cl_name, cl_surname, cl_email))
        conn.commit()

In [55]:
def phone(cl_id, num):
    cur.execute("""
    INSERT INTO phone_numbers(client_id, number) VALUES (%s, %s);
    """, (cl_id, num))
    conn.commit()

In [56]:
def edit(client_id, column, value):
    cur.execute(f"""
    UPDATE clients SET {column}=%s WHERE id=%s;
    """, (value, client_id))
    conn.commit()

In [57]:
def del_phone(cl_id, num):
    cur.execute(f"""
    DELETE FROM phone_numbers WHERE client_id=%s AND number=%s;
    """, (cl_id, num))
    conn.commit()

In [85]:
def find(table, column, info):
        cur.execute(f"""
        SELECT c.name, c.surname, c.email, ph.number FROM clients c
        JOIN phone_numbers ph ON c.id=ph.client_id
        WHERE {table}.{column}=%s;
        """, (info))
        print(cur.fetchone())

In [59]:
def del_client(cl_id):
        cur.execute("""
        DELETE FROM phone_numbers WHERE client_id=%s;
        DELETE FROM clients WHERE id=%s;
        """, (cl_id, cl_id))
        conn.commit()

db_name = 'clients'
user_name = 'postgres'
passw = 'wasser'
conn = psycopg2.connect(database = db_name, user = user_name, password = passw) 
with conn.cursor() as cur:
    cur.execute("""
    DROP TABLE phone_numbers;
    DROP TABLE clients;
    """)
conn.commit()

In [76]:
with conn.cursor() as cur:
    creation()

In [77]:
with conn.cursor() as cur:
    new_client("Till", "Lindemann", 'till@gmail.com')

In [78]:
with conn.cursor() as cur:
    phone('1', '8903505')

In [79]:
with conn.cursor() as cur:
    edit('1', 'email', 'till-l@gmail.com')

In [64]:
with conn.cursor() as cur:
    del_phone('1', '8903505')

In [90]:
with conn.cursor() as cur:
    find('c','id', '1')

('Till', 'Lindemann', 'till-l@gmail.com', 8903505)


In [91]:
with conn.cursor() as cur:
    del_client('1')

In [88]:
with conn.cursor() as cur:
    conn.rollback()